In [1]:
import numpy as np
import pandas as pd
import auxil_ML as aux

In [2]:
if 1:
    import importlib
    importlib.reload(aux);


In [52]:
#cat_fn = 'data/3FGL_prob_catalog_3class.csv'
cat_fn = '../ML_3FGL_paper_extra/data/python files/catas/3fgl_multi_cata_newfeats.csv'
cat_df3 = pd.read_csv(cat_fn, index_col=0)
cat_fn = 'data/3FGL_prob_catalog_2class.csv'
cat_df2 = pd.read_csv(cat_fn, index_col=0)

folder = '../ML_3FGL_paper/SOM'
cat_fn = '%s/3FGL_prob_catalog.csv' % folder
cat_df_3fgl = pd.read_csv(cat_fn, index_col=0)
cat_fn = '%s/4FGL-DR2_prob_catalog.csv' % folder
cat_df_4fgl = pd.read_csv(cat_fn, index_col=0)

In [53]:
classes = ['AGN', 'PSR', 'OTHER']
algs = ['RF', 'BDT', 'NN', 'LR']

In [54]:
assoc_3FGL_in_4FGL_mask = [(type(name) is str) for name in list(cat_df_4fgl['ASSOC_FGL'])]
index = [name for name in cat_df_4fgl.index[assoc_3FGL_in_4FGL_mask] 
         if cat_df_4fgl['ASSOC_FGL'].loc[name].startswith('3FGL') 
         and cat_df_4fgl['ASSOC_FGL'].loc[name] in cat_df_3fgl.index
         and cat_df_3fgl['Category_3FGL'].loc[cat_df_4fgl['ASSOC_FGL'].loc[name]] == 'UNAS']
index = [name for name in index if cat_df_4fgl['Category_4FGL'].loc[name] != 'UNAS']
assoc_3FGL_in_4FGL_df = pd.DataFrame(index=cat_df_4fgl['ASSOC_FGL'][index])
assoc_3FGL_in_4FGL_df['Category_4FGL'] = np.array(cat_df_4fgl['Category_4FGL'].loc[index])
psr_mask_4FGL = (assoc_3FGL_in_4FGL_df['Category_4FGL'] == 'PSR')
agn_mask_4FGL = (assoc_3FGL_in_4FGL_df['Category_4FGL'] == 'AGN')
other_mask_4FGL = (assoc_3FGL_in_4FGL_df['Category_4FGL'] == 'OTHER')


In [55]:
len(index)

339

In [56]:
cat_df_3fgl_i = cat_df_3fgl.loc[assoc_3FGL_in_4FGL_df.index]
cat_df3_i = cat_df3.loc[assoc_3FGL_in_4FGL_df.index]
cat_df2_i = cat_df2.loc[assoc_3FGL_in_4FGL_df.index]

In [57]:
psr_mask_prob = (cat_df_3fgl_i['Category_Prob'] == 'PSR')
agn_mask_prob = (cat_df_3fgl_i['Category_Prob'] == 'AGN')
mix_mask_prob = (cat_df_3fgl_i['Category_Prob'] == 'MIXED')

psr_mask_prob2 = (cat_df2_i['Category_Prob'] == 'PSR')
agn_mask_prob2 = (cat_df2_i['Category_Prob'] == 'AGN')
mix_mask_prob2 = (cat_df2_i['Category_Prob'] == 'MIXED')

psr_mask_prob3 = (cat_df3_i['Category_Prob'] == 'PSR')
agn_mask_prob3 = (cat_df3_i['Category_Prob'] == 'AGN')
other_mask_prob3 = (cat_df3_i['Category_Prob'] == 'OTHER')
mix_mask_prob3 = (cat_df3_i['Category_Prob'] == 'MIXED')

KeyError: 'Category_Prob'

In [58]:
print('2-class classification in paper')
print('# PSR classified as PSR', np.sum(psr_mask_4FGL & psr_mask_prob))
print('# PSR classified as AGN', np.sum(psr_mask_4FGL & agn_mask_prob))
print('# PSR classified as mixed', np.sum(psr_mask_4FGL & mix_mask_prob))
print('# AGN classified as AGN', np.sum(agn_mask_4FGL & agn_mask_prob))
print('# AGN classified as PSR', np.sum(agn_mask_4FGL & psr_mask_prob))
print('# AGN classified as mixed', np.sum(agn_mask_4FGL & mix_mask_prob))
print('# OTHER classified as AGN', np.sum(other_mask_4FGL & agn_mask_prob))
print('# OTHER classified as PSR', np.sum(other_mask_4FGL & psr_mask_prob))
print('# OTHER classified as mixed', np.sum(other_mask_4FGL & mix_mask_prob))

2-class classification in paper
# PSR classified as PSR 23
# PSR classified as AGN 10
# PSR classified as mixed 12
# AGN classified as AGN 222
# AGN classified as PSR 5
# AGN classified as mixed 30
# OTHER classified as AGN 8
# OTHER classified as PSR 6
# OTHER classified as mixed 23


In [59]:
print('2-class classification')
print('# PSR classified as PSR', np.sum(psr_mask_4FGL & psr_mask_prob2 ))
print('# PSR classified as AGN', np.sum(psr_mask_4FGL & agn_mask_prob2 ))
print('# PSR classified as mixed', np.sum(psr_mask_4FGL & mix_mask_prob2 ))
print('# AGN classified as AGN', np.sum(agn_mask_4FGL & agn_mask_prob2 ))
print('# AGN classified as PSR', np.sum(agn_mask_4FGL & psr_mask_prob2 ))
print('# AGN classified as mixed', np.sum(agn_mask_4FGL & mix_mask_prob2 ))
print('# OTHER classified as AGN', np.sum(other_mask_4FGL & agn_mask_prob2))
print('# OTHER classified as PSR', np.sum(other_mask_4FGL & psr_mask_prob2))
print('# OTHER classified as mixed', np.sum(other_mask_4FGL & mix_mask_prob2))

2-class classification
# PSR classified as PSR 23
# PSR classified as AGN 16
# PSR classified as mixed 6
# AGN classified as AGN 243
# AGN classified as PSR 5
# AGN classified as mixed 9
# OTHER classified as AGN 14
# OTHER classified as PSR 7
# OTHER classified as mixed 16


In [60]:
print('3-class classification')
print('# PSR classified as PSR', np.sum(psr_mask_4FGL & psr_mask_prob3 ))
print('# PSR classified as AGN', np.sum(psr_mask_4FGL & agn_mask_prob3 ))
print('# PSR classified as OTHER', np.sum(psr_mask_4FGL & other_mask_prob3 ))
print('# PSR classified as mixed', np.sum(psr_mask_4FGL & mix_mask_prob3 ))
print('# AGN classified as AGN', np.sum(agn_mask_4FGL & agn_mask_prob3 ))
print('# AGN classified as PSR', np.sum(agn_mask_4FGL & psr_mask_prob3 ))
print('# AGN classified as OTHER', np.sum(agn_mask_4FGL & other_mask_prob3 ))
print('# AGN classified as mixed', np.sum(agn_mask_4FGL & mix_mask_prob3 ))
print('# OTHER classified as AGN', np.sum(other_mask_4FGL & agn_mask_prob3))
print('# OTHER classified as PSR', np.sum(other_mask_4FGL & psr_mask_prob3))
print('# OTHER classified as OTHER', np.sum(other_mask_4FGL & other_mask_prob3))
print('# OTHER classified as mixed', np.sum(other_mask_4FGL & mix_mask_prob3))

3-class classification
# PSR classified as PSR 20
# PSR classified as AGN 15
# PSR classified as OTHER 0
# PSR classified as mixed 10
# AGN classified as AGN 243
# AGN classified as PSR 3
# AGN classified as OTHER 0
# AGN classified as mixed 11
# OTHER classified as AGN 10
# OTHER classified as PSR 7
# OTHER classified as OTHER 6
# OTHER classified as mixed 14


In [61]:
cat_df_3fgl_i

,GLON,GLAT,ln(Energy_Flux100),ln(Unc_Energy_Flux100),ln(Signif_Curve),ln(Variability_Index),500MeV_Index,HR12,HR23,HR34,...,PSR_NN_STD_O,AGN_RF_O,AGN_RF_STD_O,PSR_RF_O,PSR_RF_STD_O,AGN_LR_O,AGN_LR_STD_O,PSR_LR_O,PSR_LR_STD_O,Category_Prob
ASSOC_FGL,,,,,,,,,,,,,,,,,,,,,
3FGL J0002.0-6722,310.138920,-49.061836,-26.432554,-27.904511,0.313862,3.831355,1.946061,0.091518,-0.522756,-0.422249,...,0.072763,0.994358,0.006044,0.005642,0.006044,0.974034,0.009505,0.025966,0.009505,AGN
3FGL J0004.2+0843,103.599490,-52.363342,-26.538473,-27.860311,-0.235299,3.814006,2.050442,-0.755126,-0.477670,0.000580,...,0.069112,0.925361,0.047492,0.074639,0.047492,0.994295,0.005208,0.005705,0.005208,AGN
3FGL J0006.2+0135,100.400505,-59.296867,-26.405393,-27.897549,-1.778628,3.819167,2.186900,-0.611432,-0.563140,-0.645471,...,0.067848,0.989319,0.009945,0.010681,0.009945,0.999908,0.000101,0.000092,0.000101,AGN
3FGL J0007.9+4006,113.977170,-22.007206,-26.355611,-27.811253,-0.020849,3.793005,2.275604,-0.236408,-0.785758,-0.459872,...,0.064161,0.976730,0.021334,0.023270,0.021334,0.984549,0.007552,0.015451,0.007552,AGN
3FGL J0008.3+1456,107.678590,-46.658096,-26.043447,-27.812366,0.449795,3.539210,2.269549,-0.494926,-0.545080,-0.666568,...,0.133714,0.883812,0.082757,0.116188,0.082757,0.613692,0.084189,0.386308,0.084189,AGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3FGL J1855.6+0753,40.430542,2.657492,-24.114420,-26.370483,0.829996,3.433813,2.601826,-0.714195,-0.782879,-0.610913,...,0.141321,0.392464,0.114526,0.607536,0.114526,0.068482,0.023215,0.931518,0.023215,MIXED
3FGL J2004.4+3338,70.671234,1.185360,-23.868170,-26.605729,1.214844,3.917865,2.296311,-0.502915,-0.698934,-0.649789,...,0.157375,0.572448,0.097583,0.427552,0.097583,0.179261,0.051608,0.820739,0.051608,MIXED
3FGL J2058.0+4347,85.006134,-1.229651,-24.140762,-26.781395,0.417416,3.763118,2.920238,-0.725957,-0.861170,-0.999806,...,0.106712,0.731951,0.090436,0.268049,0.090436,0.981365,0.036243,0.018635,0.036243,AGN


In [62]:
true_labels = assoc_3FGL_in_4FGL_df['Category_4FGL']
classif_dict = {'paper': cat_df_3fgl_i, '2 class': cat_df2_i, '3 class': cat_df3_i}
for cname, cdf in classif_dict.items():
    print(cname)
    if cname == 'paper':
        algs_loc = algs + [alg + '_O' for alg in algs]
    else:
        algs_loc = algs
    for alg in algs_loc:
        algs_loc = [alg]
        prob_labels = aux.get_prob_class(cdf, algs_loc, classes)['Category_Prob']
        acc_all = 100 * aux.accuracy(true_labels, prob_labels)
        print('%s, %.2f percent' % (alg, acc_all))

paper
RF, 81.12 percent
BDT, 80.53 percent
NN, 80.24 percent
LR, 80.53 percent
RF_O, 79.65 percent
BDT_O, 81.71 percent
NN_O, 77.58 percent
LR_O, 75.81 percent
2 class
RF, 81.42 percent
BDT, 81.71 percent
NN, 80.83 percent
LR, 79.65 percent
3 class
RF, 84.66 percent
BDT, 83.48 percent
NN, 82.60 percent
LR, 79.65 percent


In [200]:
cat_df2_i

,AGN_RF,PSR_RF,n,AGN_BDT,PSR_BDT,AGN_NN,PSR_NN,AGN_LR,PSR_LR,Category_Prob,Category_3FGL
ASSOC_FGL,,,,,,,,,,,
3FGL J0002.0-6722,0.998707,0.001293,38,1.000000,6.194107e-10,0.994900,0.005100,0.998169,1.831446e-03,AGN,UNAS
3FGL J0002.6+6218,0.031480,0.968520,38,0.000011,9.999885e-01,0.018046,0.981954,0.032603,9.673966e-01,PSR,UNAS
3FGL J0004.2+0843,0.912905,0.087095,38,1.000000,4.102354e-07,0.996602,0.003398,0.999125,8.751577e-04,AGN,UNAS
3FGL J0006.2+0135,0.996868,0.003132,38,1.000000,7.224723e-10,0.998612,0.001388,1.000000,1.676263e-07,AGN,UNAS
3FGL J0007.9+4006,0.997235,0.002765,38,1.000000,6.642705e-08,0.993507,0.006493,0.999132,8.678554e-04,AGN,UNAS
...,...,...,...,...,...,...,...,...,...,...,...
3FGL J1843.0-0359c,0.146477,0.853523,38,0.000196,9.998035e-01,0.117759,0.882241,0.210846,7.891539e-01,PSR,UNAS
3FGL J1850.5-0024,0.389261,0.610739,38,0.224297,7.757034e-01,0.442533,0.557467,0.613338,3.866616e-01,MIXED,UNAS
3FGL J1855.6+0753,0.529337,0.470663,38,0.260910,7.390897e-01,0.298355,0.701645,0.516406,4.835936e-01,MIXED,UNAS
